<a href="https://colab.research.google.com/github/farmountain/SmartGlass-AI-Agent/blob/main/colab_notebooks/Session13_Healthcare_UseCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏥 Week 13: Use Case — Healthcare Smart Glass Assistant
Simulate a Ray-Ban Meta smart glass assistant in a hospital or clinical setting. This notebook shows how to:

- Capture and transcribe real-time doctor-patient interaction
- Apply medical keyword extraction with spaCy
- Auto-translate key terms
- Suggest follow-up actions using GPT-2 or LLaMA2


In [ ]:
# ✅ Step 0: RESTART RUNTIME first to clear all memory (important)
# After restarting, run everything below

# ✅ Step 1: Install all dependencies (no SciSpacy)
!pip install -q openai-whisper gtts pydub googletrans==4.0.0-rc1 spacy transformers

# ✅ Optional (for fallback noun-chunk extraction)
!python -m spacy download en_core_web_sm


# ✅ Step 3: Google Translate
!pip install -U googletrans==4.0.0-rc1

# ✅ Step 4: Medical NLP
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")



In [ ]:
import whisper
from gtts import gTTS
from pydub import AudioSegment
from IPython.display import Audio, display
import spacy
import torch
from transformers import pipeline
from googletrans import Translator

# ✅ 1. Simulate Audio Input (patient voice)
tts = gTTS("Patient reports chest pain for two days and dizziness.", lang='en')
tts.save("session_audio.mp3")
sound = AudioSegment.from_file("session_audio.mp3")
sound.export("session_audio.wav", format="wav")

# ✅ 2. Whisper Transcription
asr_model = whisper.load_model("base")  # use "tiny" for CPU speed
result = asr_model.transcribe("session_audio.wav")
print("🩺 Transcribed Text:", result['text'])

# ✅ 3. Medical NER Extraction via BioBERT
try:
    print("🔍 Loading BioBERT Medical NER model...")
    ner_pipeline = pipeline("ner", model="kamalkraj/BioBERT-NER", aggregation_strategy="simple")
    ner_results = ner_pipeline(result['text'])
    keywords = [res['word'] for res in ner_results if res['score'] > 0.7]
    print("🧠 Extracted Medical Keywords:", keywords)
except Exception as e:
    print(f"⚠️ BioBERT model failed: {e}")
    keywords = []

# ✅ 4. Fallback: use spaCy noun chunks if no keywords found
if not keywords:
    print("🔁 Falling back to noun-chunk extraction...")
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(result['text'])
    keywords = [chunk.text for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]
    print("🧩 Extracted noun chunks:", keywords)

# ✅ 5. Translate medical keywords to Chinese
translator = Translator()
zh_keywords = []
for word in keywords:
    translated = translator.translate(word, dest='zh-CN')
    zh_keywords.append(f"{word} → {translated.text}")
print("🌐 Translated Keywords:", zh_keywords)

# ✅ 6. Speak the Chinese translation
chinese_summary = "，".join([t.split(" → ")[1] for t in zh_keywords])
if chinese_summary.strip():
    tts_zh = gTTS(chinese_summary, lang='zh-CN')
    tts_zh.save("translated_keywords.mp3")
    display(Audio("translated_keywords.mp3"))
else:
    print("⚠️ No content to speak.")
